# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from IPython.display import display

In [2]:
data = pd.read_csv('/datasets/insurance.csv')
display(data, data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


None

In [3]:
data = data.astype({'Возраст':'int', 'Зарплата':'int'})

## Вывод

Имеем таблицу, пропусков нет, 5 столбцов, 5000 строк, изменил тип данных у 2-х столбцов на int.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответьте на вопрос и обоснуйте решение.**

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

a. Изменится. Приведите примеры матриц.

b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

**Ответ:** b. Не изменится.

**Обоснование:** 

Исходная формула выглядит так:

$$ w' = ((XP)^T XP)^{-1} (XP)^T y $$

Начинаю преобразования с раскрытия скобок:

$$ (XP)^T = X^T P^T $$

$$ (X^T P^T XP)^{-1} = (P^T)^{-1} (X^T XP)^{-1} = (P^T)^{-1} (X^T X)^{-1} P^{-1} $$

После раскрытия скобок получаю формулу вида:

$$ w' = (P^T)^{-1} (X^T X)^{-1} P^{-1} X^T P^T y $$

Умножение обратной и обратимой матриц даёт единичную матрицу:

$$ (P^T)^{-1} P^T = E^T = E $$

Теперь формула выглядит так:

$$ w' = E (X^T X)^{-1} P^{-1} X^T y $$

Вывожу $y$ из формулы обучения:

$$ y = w ((X^T X)^{-1} X^T)^{-1} $$

Подставляю $y$ в формулу:

$$ w' = (X^T X)^{-1} P^{-1} X^T w ((X^T X)^{-1} X^T)^{-1} $$

Сокращаю:

$$ w' = (X^T X)^{-1} P^{-1} X^T w (X^T X) (X^{-1})^T $$

Получаю формулу:

$$ w' = P^{-1} w $$

Формула вектора предсказаний:

$$ a' = X P w' $$

Подставляю $w'$ в формулу вектора предсказаний:

$$ a' = X P P^{-1} w $$

В итоге:

$$ a = X w = a' $$

## Алгоритм преобразования

Предложите алгоритм преобразования данных для решения задачи. 

Обоснуйте, почему качество линейной регрессии не поменяется.

**Алгоритм**

1. Генерирую рандомную матрицу размера 4*4
2. Проверяю на обратимость сгенерированную матрицу через try/except
3. Домножау матрицу признаков на сгенерированную матрицу.
4. Определяю R2 для признаков и признаков, умноженных на сгенерированную матрицу
5. Сравниваю метрики с помощью isclose

**Обоснование**

Суть алгоритма заключается в том, что я подставляю в формулу весов признаки, умноженные на обратимую матрицу, а затем для получения предсказаний я умножаю признаки на обратную матрицу, а из формулы $ P P^{-1} = E$ и формулы $P E = P$ получается, что вектор предсказаний высчитывается на основе изначальных признаков, тоесть $ a = X w = a' $

## Проверка алгоритма

Запрограммируйте этот алгоритм, применив матричные операции. 

Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. 

Примените метрику R2.

Обучаю модель линейной регрессии на признаках:

In [4]:
features = data.drop(['Страховые выплаты'], axis=1) 
target = data['Страховые выплаты']

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

model = LinearRegression()
model.fit(features_train, target_train)
predictions_valid_model1 = model.predict(features_valid)

result = r2_score(target_valid, predictions_valid_model1)
print("R2 модели линейной регрессии на валидационной выборке:", result)

R2 модели линейной регрессии на валидационной выборке: 0.4352275684083349


Обучаю модель линейной регрессии на признаках, умноженных на обратимую матрицу (чтоб получить обратимую матрицу, я умножил матрицу признаков на саму себя транспонированную):

In [5]:
features = data.drop(['Страховые выплаты'], axis=1) 
P = np.random.normal(size=(4,4))
try:
    P_inv = np.linalg.inv(P)
except:
    print('необратимая матрица')
best_features = features.dot(P)
target = data['Страховые выплаты']

features_train, features_valid, target_train, target_valid = train_test_split(
best_features, target, test_size=0.25, random_state=12345)

model = LinearRegression()
model.fit(features_train, target_train)
predictions_valid_model2 = model.predict(features_valid)

result = r2_score(target_valid, predictions_valid_model2)
print("R2 модели линейной регрессии на валидационной выборке c признаками, умноженными на обратимую матрицу:", result)


R2 модели линейной регрессии на валидационной выборке c признаками, умноженными на обратимую матрицу: 0.43522756840796795


Проверка результатов на сходство:

In [6]:
np.isclose(r2_score(target_valid, predictions_valid_model1), r2_score(target_valid, predictions_valid_model2))

True

## Вывод

В данном проекте я выполнил кодирование признаков путем умножения на обратимую матрицу и убедился, что качество модели линейной регрессии не ухудшилось.